In [39]:
from statistics import correlation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

## Metrics to use
Separate metrics into top 1 and top 5 category
Calculate accuracy precision and recall for both categories. Also add F1 score

In [40]:
pred_df = pd.read_csv('data/pred_results.csv')
correlations = pd.read_csv('data/correlations.csv')

In [41]:
content_to_topic = {'content_id': [], 'topic_id': []}
for i, row in correlations.iterrows():
    for content_id in row['content_ids'].split(' '):
        content_to_topic['content_id'].append(content_id)
        content_to_topic['topic_id'].append(row['topic_id'])
content_to_topic = pd.DataFrame(content_to_topic)
content_to_topic = content_to_topic[content_to_topic['content_id'].isin(pred_df['content_id'])]

In [42]:
pred_df = pred_df.set_index(['topic_id', 'content_id'])
content_to_topic['label'] = 1
content_to_topic = content_to_topic.set_index(['topic_id', 'content_id'])

## TOP 1

In [43]:
content_to_topic.reset_index()['content_id'].value_counts()

content_id
c_8ad113a7a801    24
c_15a634632521    14
c_04fbee306e50    13
c_b4429618cba1    13
c_76a17cfb9d87    12
                  ..
c_a662335f86a4     1
c_e28009ea492f     1
c_c733b0afd4f8     1
c_9022950ceacc     1
c_10157e74fc3a     1
Name: count, Length: 1000, dtype: int64

In [44]:
top_1 = pred_df.reset_index().groupby('content_id')['topic_id'].first()
joined = content_to_topic.join(top_1, on=['content_id',], how='left')
joined = joined.rename(columns={'topic_id': 'pred_topic'})
joined = joined.reset_index()
joined['label'] = (joined['topic_id'] == joined['pred_topic']) * 1.0

top1_acc = (joined.groupby('content_id')['label'].mean() > 0).mean()
print('Top 1 Accuracy', top1_acc)
random_prec = (content_to_topic.groupby('content_id')['label'].count() / len(correlations)).mean()
print('Random accuracy (baseline)', random_prec)

Top 1 Accuracy 0.232
Random accuracy (baseline) 2.9617829217939747e-05


## TOP 5 metrics against all topic content

In [47]:
true = content_to_topic.index
pred = pred_df.index
recall = true.isin(pred).mean()
precision = pred.isin(true).mean()
f1 = 2 * precision * recall / (precision + recall)
print('Global metrics')
print('Recall:', recall)
print('Precision:', precision)
print('F1:', f1)

Global metrics
Recall: 0.2650933040614709
Precision: 0.0966
F1: 0.14160070360598065
